In [1]:
from sqlalchemy import create_engine

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float

In [2]:
from sqlalchemy import create_engine
database_path = "FPA_FOD_20170508.sqlite"

In [3]:
engine = create_engine(f"sqlite:///{database_path}")
Base.metadata.create_all(engine)

In [4]:
# NOTE: to print the tables of the database
print (engine.table_names())

['ElementaryGeometries', 'Fires', 'KNN', 'NWCG_UnitIDActive_20170109', 'SpatialIndex', 'geometry_columns', 'geometry_columns_auth', 'geometry_columns_field_infos', 'geometry_columns_statistics', 'geometry_columns_time', 'idx_Fires_Shape', 'idx_Fires_Shape_node', 'idx_Fires_Shape_parent', 'idx_Fires_Shape_rowid', 'spatial_ref_sys', 'spatial_ref_sys_aux', 'spatialite_history', 'sql_statements_log', 'sqlite_sequence', 'views_geometry_columns', 'views_geometry_columns_auth', 'views_geometry_columns_field_infos', 'views_geometry_columns_statistics', 'virts_geometry_columns', 'virts_geometry_columns_auth', 'virts_geometry_columns_field_infos', 'virts_geometry_columns_statistics']


In [5]:
from sqlalchemy.orm import Session
session = Session(bind=engine)


# NOTE: use dataframe to see column names
import pandas as pd
fire_df = pd.read_sql('select * from Fires', engine)
fire_df.sample(20)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
342159,342160,350796,W-125367,FED,DOI-WFMI,BLM,USORPRD,Prineville District,ORPRD,Prineville District,...,A,44.298500,-121.343100,1.0,BLM,OR,Deschutes,017,Deschutes,b'\x00\x01\xad\x10\x00\x00|\xd0\xb3Y\xf5U^\xc0...
1693130,1693131,201863797,TFS-TXFD2013-459848,NONFED,ST-TXTXS,ST/C&L,USTXTXS,Texas A & M Forest Service,TXVFD,Honey Grove Volunteer Fire,...,B,33.608366,-95.962050,8.0,PRIVATE,TX,Fannin,147,Fannin,b'\x00\x01\xad\x10\x00\x00x\xc7):\x92\xfdW\xc0...
769896,769897,871945,ODF-2710,NONFED,ST-ORORS,ST/C&L,USOR955S,Sisters Unit,ORODF-955,Sisters Unit,...,A,43.966290,-121.462850,7.0,STATE,OR,17,017,Deschutes,b'\x00\x01\xad\x10\x00\x00\x04=\x9bU\x9f]^\xc0...
523211,523212,563412,SFO-NC0395-01114,NONFED,ST-NASF,ST/C&L,USNCNCS,North Carolina Forest Service,NCNCS,North Carolina Division of Forest Resources,...,B,35.772930,-81.241550,14.0,MISSING/NOT SPECIFIED,NC,Catawba,035,Catawba,b'\x00\x01\xad\x10\x00\x00P\x96!\x8euOT\xc0x\x...
483972,483973,520857,SFO-ME0279-07220009,NONFED,ST-NASF,ST/C&L,USMEMES,Maine Forest Service,MEMES,Maine Forest Service,...,B,44.634310,-70.027440,14.0,MISSING/NOT SPECIFIED,ME,Franklin,007,Franklin,b'\x00\x01\xad\x10\x00\x00\x88\xa6\xb3\x93\xc1...
1550421,1550422,201407415,TFS-TXFD2011-336225,NONFED,ST-TXTXS,ST/C&L,USTXTXS,Texas A & M Forest Service,TXVFD,Jarret Volunteer Fire Asso.,...,B,29.295704,-98.734245,13.0,STATE OR PRIVATE,TX,Bexar,029,Bexar,b'\x00\x01\xad\x10\x00\x00\x18\x90\xbd\xde\xfd...
1321989,1321990,1800898,SFO-WV-1999-6323,NONFED,ST-NASF,ST/C&L,USWVWVS,West Virginia Division of Forestry,WVDOF,West Virginia Division of Forestry,...,B,38.127349,-82.465756,8.0,PRIVATE,WV,Wayne,099,Wayne,b'\x00\x01\xad\x10\x00\x00\x8c\xd9Y\xf4\xce\x9...
1384109,1384110,19079860,SFO-GA-WIL-32-9/28/1993-1320,NONFED,ST-GAGAS,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,Georgia Forestry Commission,...,B,33.675100,-82.522800,8.0,PRIVATE,GA,Wilkes,317,Wilkes,b'\x00\x01\xad\x10\x00\x00P\x96!\x8eu\xa1T\xc0...
211881,211882,214872,W-26129,FED,DOI-WFMI,BLM,USAZASD,Arizona Strip Field Office,AZASD,Arizona Strip Field Office,...,A,36.866600,-113.684100,1.0,BLM,AZ,None,None,None,b'\x00\x01\xad\x10\x00\x00`\xcc]K\xc8k\\\xc0`[...
793760,793761,899938,SCHMIDT_55012003,NONFED,ST-WIWIS,ST/C&L,USWIWIS,Wisconsin Department of Natural Resources,WIWIS,Wisconsin Department of Natural Resources,...,A,44.845638,-89.621599,14.0,MISSING/NOT SPECIFIED,WI,Marathon,073,Marathon,"b'\x00\x01\xad\x10\x00\x00,\xc1-H\xc8gV\xc08\x..."


In [9]:
# Create a filtered dataframe from specific columns
fire_cols = ["FIRE_SIZE_CLASS", "LATITUDE", "LONGITUDE", "STATE","FIPS_CODE","FIPS_NAME"]
fire_tf= fire_df[fire_cols].copy()

# Rename the column headers
fire_tf = fire_tf.rename(columns={"FIRE_SIZE_CLASS": "class",
                                  "LATITUDE":"lat",
                                  "LONGITUDE":"long",
                                  "STATE":"state",
                                  "FIPS_CODE":"fcode",
                                  "FIPS_NAME":"fcounty"})

# Clean the data by dropping duplicates and setting the index
#fire_tf.drop_duplicates("id", inplace=True)
#fire_tf.set_index("id", inplace=True)

fire_tf.head()


,class,lat,long,state,fcode,fcounty
0,A,40.036944,-121.005833,CA,063,Plumas
1,A,38.933056,-120.404444,CA,061,Placer
2,A,38.984167,-120.735556,CA,017,El Dorado
3,A,38.559167,-119.913333,CA,003,Alpine
4,A,38.559167,-119.933056,CA,003,Alpine


In [4]:
# create your shark class
class Fires(Base):
    __tablename__ = 'fires'
    id = Column(Integer, primary_key=True)
    case_number = Column(String)
    date = Column(String)
    year = Column(Integer)
    type = Column(String)
    country = Column(String)
    area = Column(String)
    location= Column(String)
    activity = Column(String)
    name = Column(String)
    sex = Column(String)
    age = Column(Integer)
    injury = Column(String)
    fatal_y_n = Column(String)
    time = Column(String)
    species = Column(String)
    investigator_or_source = Column(String)
    pdf = Column(String)
    original_order = Column(Integer)

In [5]:
# print all locations of shark attacks
attacks = session.query(Sharks) 
for attack in attacks:
    print(attack.location)

Point Casuarina, Bunbury
Bateman's Bay
Ponce Inlet, Volusia County
Pulau Payar Island
Alenuihaha Channel
The Natural Jetty, Rottnest Island
Near Albany
Bourail
Off Cape Point
Taranaki
Avalon
Honokowai, Maui


In [6]:
# find the number of provoked attacks
provoked = session.query(Sharks).filter_by(type='Provoked').count()
print(provoked)

12


In [7]:
# find the number of attacks in USA
usa = session.query(Sharks).filter_by(country='USA').count()
print(usa)

28


In [8]:
# find the number of attacks in 2017
year_2017 = session.query(Sharks).filter_by(year="2017").count()
print(year_2017)

3


In [9]:
# find the number of attacks while surfing
surfing = session.query(Sharks).filter_by(activity='Surfing').count()
print(surfing)

34


In [10]:
# find the number of fatal attacks
fatal = session.query(Sharks).filter_by(fatal_y_n='Y').count()
print(fatal)

12


In [11]:
# find the number of fatal attacks while surfing
fatal_surfing = session.query(Sharks).\
    filter_by(fatal_y_n='Y').\
    filter(Sharks.area == "Eastern Cape Province").count()
print(fatal_surfing)

2


In [12]:
# find the number of fatal attacks in 2017 in Australia
fatal_surfing = session.query(Sharks).\
    filter_by(fatal_y_n='Y').\
    filter(Sharks.country == "MOZAMBIQUE").\
    filter(Sharks.activity == 'Spearfishing').count()
print(fatal_surfing)

1
